In [2]:
%pylab
import PyDSTool as dst
import numpy as np

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [93]:
##### Cartisian 
# x_dot_i = gamma*(mu - r_i^2)*x_i - omega_i*y_i + eps*F + tau*sin(R_i - theta_i)
# y_dot_i = gamma*(mu - r_i^2)*y_i + omega_i*x_i
# omega_dot_i = -esp*F*(y_i/r_i)
# alpha_dot_i = nu*x_i*F
# theta_dot_i = sin(R_i - sgn(x_i)*cos^-1(-(y_i/r_i)))

# r = sqrt(x_i^2 + y_i^2)
# R_i = (omega_i/omega_0)*sgn(x_0)*cos^-1(-(y_0/sqrt(x_0^2 + y_0^2)))
# F = signal [P_teach - sum(alpha_i*x_i)]
# P_teach = 0.8*sin(15*t)+cos(30*t)-1.4*sin(45*t)-0.5*cos(60*t)


DSargs = dst.args()
DSargs.name = 'Adaptive_multi_Hopf_Oscillator_Test'
DSargs.pars = {'gamma1': 8.,
               'mu1': 1.,
               'tau1': 2.,
               'nu1': 0.5,
               'esp': 0.9,
               }
DSargs.varspecs = {}
DSargs.ics = {}

P_teach = '(0.8*sin(2.*t)+cos(8.*t)-1.4*sin(15.*t)-0.5*cos(25.*t))'

num_cpgs = 4
temp_string = '(alpha_0*x_0'
for idx in range(num_cpgs-1):
    if(idx+1 == num_cpgs-1):
        temp_string = temp_string + ' + alpha_{i}*x_{i})'.format(i = idx+1)
    else:
        temp_string = temp_string + ' + alpha_{i}*x_{i}'.format(i = idx+1)
    
for idx in range(num_cpgs):
    if(idx == 0):
        temp_varspecs = {'x_%d'%idx: 'gamma1*(mu1 - (sqrt(x_{i}^2 + y_{i}^2))^2)*x_{i} - omega_{i}*y_{i} + esp*({P} - {F}) + tau1*sin((omega_{i}/omega_0)*sign(x_0)*cos(-(y_0/sqrt(x_0^2 + y_0^2)))^-1 - theta_{i})'.format(i = idx, P = P_teach, F = temp_string),
                         'y_%d'%idx: 'gamma1*(mu1 - (sqrt(x_{i}^2 + y_{i}^2))^2)*y_{i} + omega_{i}*x_{i}'.format(i = idx),
                         'omega_%d'%idx: '-esp*({P} - {F})*(y_{i}/sqrt(x_{i}^2 + y_{i}^2))'.format(i = idx, P = P_teach, F = temp_string),
                         'alpha_%d'%idx: 'nu1*x_{i}*({P} - {F})'.format(i = idx, P = P_teach, F = temp_string),
                         'theta_%d'%idx: '0',
                        }

        DSargs.varspecs.update(temp_varspecs)
    else:
        temp_varspecs = {'x_%d'%idx: 'gamma1*(mu1 - (sqrt(x_{i}^2 + y_{i}^2))^2)*x_{i} - omega_{i}*y_{i} + esp*({P} - {F}) + tau1*sin((omega_{i}/omega_0)*sign(x_0)*cos(-(y_0/sqrt(x_0^2 + y_0^2)))^-1 - theta_{i})'.format(i = idx, P = P_teach, F = temp_string),
                         'y_%d'%idx: 'gamma1*(mu1 - (sqrt(x_{i}^2 + y_{i}^2))^2)*y_{i} + omega_{i}*x_{i}'.format(i = idx),
                         'omega_%d'%idx: '-esp*({P} - {F})*(y_{i}/sqrt(x_{i}^2 + y_{i}^2))'.format(i = idx, P = P_teach, F = temp_string),
                         'alpha_%d'%idx: 'nu1*x_{i}*({P} - {F})'.format(i = idx, P = P_teach, F = temp_string),
                         'theta_%d'%idx: 'sin((omega_{i}/omega_0)*sign(x_0)*cos(-(y_0/sqrt(x_0^2 + y_0^2)))^-1 - sign(x_{i})*cos(-(y_{i}/sqrt(x_{i}^2 + y_{i}^2)))^-1)'.format(i = idx),
                        }

        DSargs.varspecs.update(temp_varspecs)
    
for idx in range(len(DSargs.varspecs.keys())):
    if('x' in DSargs.varspecs.keys()[idx]):
        DSargs.ics.update({'%s'%DSargs.varspecs.keys()[idx]: 1.})
    elif('omega' in DSargs.varspecs.keys()[idx]):
        DSargs.ics.update({'%s'%DSargs.varspecs.keys()[idx]: idx*2.})
    else:
        DSargs.ics.update({'%s'%DSargs.varspecs.keys()[idx]: 0.})
    
DSargs.tdomain = [0,500]
solver = dst.Generator.Vode_ODEsystem(DSargs) #Vode_ODEsystem(DSargs)
traj = solver.compute('CPG')
pts = traj.sample(dt=1)

In [94]:
pts = traj.sample(dt=0.001)

In [102]:
plt.plot(pts['t'], (pts['alpha_0']*pts['x_0'] + pts['alpha_1']*pts['x_1'] + pts['alpha_2']*pts['x_2'] + pts['alpha_3']*pts['x_3']))
#plt.plot(pts['t'], pts['y'])
plt.xlabel('time')
#plt.xlim([999,1000])
#plt.ylim([-8,8])
plt.title(solver.name)

In [96]:
plt.plot(pts['t'], pts['x_0'])
plt.xlim([0,250])

(0, 250)

In [100]:
plt.plot(pts['t'], pts['omega_3'])
plt.xlim([0,250])

(0, 250)